In [1]:
from typing import Tuple
import numpy as np
from itertools import combinations

## Lexicographic pattern

In [2]:
from numba import njit
n_det = 24

n_comb = int((n_det * (n_det - 1) * (n_det - 2)) / 6)
crater_triads = np.empty((n_comb, 3), np.int)
for i, el in enumerate(combinations(np.arange(n_det), 3)):
    crater_triads[i] = el

crater_triads

C:\Users\w_dop\miniconda3\envs\crater-detection\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


array([[ 0,  1,  2],
       [ 0,  1,  3],
       [ 0,  1,  4],
       ...,
       [20, 21, 23],
       [20, 22, 23],
       [21, 22, 23]])

## Enhanced Pattern Shifting
Arnas, D., Fialho, M. A. A., & Mortari, D. (2017). Fast and robust kernel generators for star trackers. Acta Astronautica,
134 (August 2016), 291–302. https://doi.org/10.1016/j.actaastro.2017.02.016

In [10]:
# Verify with p. 292 from https://doi.org/10.1016/j.actaastro.2017.02.016
@njit
def enhanced_pattern_shifting(n) -> Tuple[int, int, int]:
    if n < 3:
        raise ValueError("Number of detections must be equal or higher than 3!")

    for dj in range(1, n - 1):
        for dk in range(1, n - dj):
            for ii in range(1, 4):
                for i in range(ii, n - dj - dk + 1, 3):
                    j = i + dj
                    k = j + dk
                    yield i - 1, j - 1, k - 1

In [4]:
crater_triads = np.empty((n_comb, 3), np.uint32)
for it, (i, j, k) in enumerate(enhanced_pattern_shifting(9)):
    print((i, j, k))

(0, 1, 2)
(3, 4, 5)
(6, 7, 8)
(1, 2, 3)
(4, 5, 6)
(2, 3, 4)
(5, 6, 7)
(0, 1, 3)
(3, 4, 6)
(1, 2, 4)
(4, 5, 7)
(2, 3, 5)
(5, 6, 8)
(0, 1, 4)
(3, 4, 7)
(1, 2, 5)
(4, 5, 8)
(2, 3, 6)
(0, 1, 5)
(3, 4, 8)
(1, 2, 6)
(2, 3, 7)
(0, 1, 6)
(1, 2, 7)
(2, 3, 8)
(0, 1, 7)
(1, 2, 8)
(0, 1, 8)
(0, 2, 3)
(3, 5, 6)
(1, 3, 4)
(4, 6, 7)
(2, 4, 5)
(5, 7, 8)
(0, 2, 4)
(3, 5, 7)
(1, 3, 5)
(4, 6, 8)
(2, 4, 6)
(0, 2, 5)
(3, 5, 8)
(1, 3, 6)
(2, 4, 7)
(0, 2, 6)
(1, 3, 7)
(2, 4, 8)
(0, 2, 7)
(1, 3, 8)
(0, 2, 8)
(0, 3, 4)
(3, 6, 7)
(1, 4, 5)
(4, 7, 8)
(2, 5, 6)
(0, 3, 5)
(3, 6, 8)
(1, 4, 6)
(2, 5, 7)
(0, 3, 6)
(1, 4, 7)
(2, 5, 8)
(0, 3, 7)
(1, 4, 8)
(0, 3, 8)
(0, 4, 5)
(3, 7, 8)
(1, 5, 6)
(2, 6, 7)
(0, 4, 6)
(1, 5, 7)
(2, 6, 8)
(0, 4, 7)
(1, 5, 8)
(0, 4, 8)
(0, 5, 6)
(1, 6, 7)
(2, 7, 8)
(0, 5, 7)
(1, 6, 8)
(0, 5, 8)
(0, 6, 7)
(1, 7, 8)
(0, 6, 8)
(0, 7, 8)


In [5]:
batch_size = 30
eps_gen = enhanced_pattern_shifting(n_det)

for i in range(2):

    for index, (i, j, k) in enumerate(eps_gen):
        print(index, (i, j, k))
        if index >= batch_size -1:
            break

0 (0, 1, 2)
1 (3, 4, 5)
2 (6, 7, 8)
3 (9, 10, 11)
4 (12, 13, 14)
5 (15, 16, 17)
6 (18, 19, 20)
7 (21, 22, 23)
8 (1, 2, 3)
9 (4, 5, 6)
10 (7, 8, 9)
11 (10, 11, 12)
12 (13, 14, 15)
13 (16, 17, 18)
14 (19, 20, 21)
15 (2, 3, 4)
16 (5, 6, 7)
17 (8, 9, 10)
18 (11, 12, 13)
19 (14, 15, 16)
20 (17, 18, 19)
21 (20, 21, 22)
22 (0, 1, 3)
23 (3, 4, 6)
24 (6, 7, 9)
25 (9, 10, 12)
26 (12, 13, 15)
27 (15, 16, 18)
28 (18, 19, 21)
29 (1, 2, 4)
0 (4, 5, 7)
1 (7, 8, 10)
2 (10, 11, 13)
3 (13, 14, 16)
4 (16, 17, 19)
5 (19, 20, 22)
6 (2, 3, 5)
7 (5, 6, 8)
8 (8, 9, 11)
9 (11, 12, 14)
10 (14, 15, 17)
11 (17, 18, 20)
12 (20, 21, 23)
13 (0, 1, 4)
14 (3, 4, 7)
15 (6, 7, 10)
16 (9, 10, 13)
17 (12, 13, 16)
18 (15, 16, 19)
19 (18, 19, 22)
20 (1, 2, 5)
21 (4, 5, 8)
22 (7, 8, 11)
23 (10, 11, 14)
24 (13, 14, 17)
25 (16, 17, 20)
26 (19, 20, 23)
27 (2, 3, 6)
28 (5, 6, 9)
29 (8, 9, 12)


In [32]:
@njit
def eps_array(n):
    n_comb = int((n * (n - 1) * (n - 2)) // 6)

    out = np.empty((n_comb, 3), np.uint32)

    for ii, (i, j, k) in enumerate(enhanced_pattern_shifting(n)):
        out[ii, 0] = i
        out[ii, 1] = j
        out[ii, 2] = k

    return out

In [37]:
%%time
crater_triads = eps_array(900)

Wall time: 1.12 s


In [38]:
crater_triads

array([[  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8],
       ...,
       [  1, 898, 899],
       [  0, 897, 899],
       [  0, 898, 899]], dtype=uint32)